In [4]:
import os 
import zipfile
import StringIO
import pandas as pd

datadir = '/Users/kanaan/Google Drive/TS-EUROTRAIN/RESULTS_QSMv3/dataframes/AHBA/ahba_data/'

donors = ['donor1', 'donor2', 'donor3', 'donor4', 'donor5', 'donor6']

In [5]:
def return_donor_geneexpression(donor):
    
    #I/O
    donor_zip  = zipfile.ZipFile(os.path.join(datadir,'%s.zip'%donor))
    
    #read microarray data. take care indexing.... should have 58692 probes
    df_microarray = pd.read_csv(StringIO.StringIO(donor_zip.read('MicroarrayExpression.csv')),
                                header= None,index_col =0 )
    
    # read probe meta-data
    df_probes = pd.read_csv(StringIO.StringIO(donor_zip.read('Probes.csv')), index_col =0 )
    del df_probes.index.name
    
    # drop unannotated genes... should end up with 48170 probes
    drop_strings = ['uncharacterized', 'non-RefSeq']
    drop_strings = ['non-RefSeq']
    drop_probes  = [i for i in df_probes.index for x in drop_strings if x in df_probes.loc[i]['gene_name'] ]
    df_probes = df_probes.drop(drop_probes, axis =0)
    
    # Calc mean probe expression for each gene
    df_gene_exp = pd.concat([df_microarray, df_probes['gene_symbol']],axis=1).groupby('gene_symbol').mean().T.reset_index().drop(['index'],axis=1)
    df_gene_exp.columns.name = None
    
    # Add metadata to dataframe
    df_meta = pd.read_csv(StringIO.StringIO(donor_zip.read('SampleAnnot.csv')))
    df = pd.concat([df_gene_exp,df_meta],axis=1).set_index('well_id')
    del df.index.name
    
    # append donot name
    df['donor'] = donor
    return df


In [6]:
df_all = []
for donor in donors:
    df = return_donor_geneexpression(donor)
    df_all.append(df)

In [7]:
AHBA = pd.concat(df_all, axis = 0)

In [8]:
MATRIX = AHBA.iloc[:,:-13]

In [9]:
AHBA.to_csv(os.path.join(datadir,'AHBA_20737.csv'))

In [10]:
AHBA.head()

,61E3.4,A1BG,A1CF,A26C1B,A2BP1,A2LD1,A2M,A2ML1,A3GALT2P,A4GALT,...,structure_acronym,structure_name,polygon_id,mri_voxel_x,mri_voxel_y,mri_voxel_z,mni_x,mni_y,mni_z,donor
594,12.257452,4.307083,1.639045,6.047821,11.502535,2.992096,7.702302,4.657058,1.675294,3.470479,...,PCLa-i,"paracentral lobule, anterior part, right, infe...",37470,87,52,116,5.9,-27.7,49.7,donor1
2985,12.971966,5.394092,2.016619,6.258501,12.314500,3.450627,8.285363,5.044534,3.596557,4.651128,...,Cl,"claustrum, right",40517,66,92,63,29.2,17.0,-2.9,donor1
2801,13.397286,6.196477,2.419608,5.442959,12.776440,3.652065,8.698773,4.777458,3.662503,4.845015,...,Cl,"claustrum, right",41516,66,81,104,28.2,-22.8,16.8,donor1
2273,13.326607,5.665946,2.473735,4.363560,12.428296,4.512593,9.333068,5.325190,2.956374,3.377773,...,LGd,"dorsal lateral geniculate nucleus, left",41473,116,94,101,-24.6,-24.6,1.3,donor1
2785,13.356659,5.087975,2.391748,4.590116,13.365749,3.558010,9.939992,5.247008,3.760852,4.446860,...,CA4,"CA4 field, right",41142,63,104,106,31.1,-31.3,-7.3,donor1
